In [7]:
# Required libraries
import pandas as pd
import requests
import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
import math
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy
from nrclex import NRCLex
import string
from transformers import AutoTokenizer, AutoModel
from collections import namedtuple
import math

# NLTK resource downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

print("NLTK resources downloaded successfully.")


NLTK resources downloaded successfully.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amirh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amirh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amirh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# Load the dataset
df = pd.read_csv('final_dataset_language.csv')
print("Initial dataset loaded:")
df.head()


Initial dataset loaded:


,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,popularity,...,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank,lyrics_length,language,unique_words
0,GBAAN7390003,"['reggae', 'ska']",i shot the sheriff,the wailers,Legend (Deluxe Edition),1984-05-01,day,spotify:track:71zvW0WZ0bap9gphp7Jhgp,71zvW0WZ0bap9gphp7Jhgp,56,...,i shot the sheriff,https://www.musixmatch.com/lyrics/The-Wailers/...,"I shot the sheriff, but I didn't shoot no depu...",0,0,0.435196,0,1577,en,126
1,USWB10001875,"['funk', 'rock']",computer blue,prince,Purple Rain,1984-06-25,day,spotify:track:3rNCkAhv9cASYco3kC9q3D,3rNCkAhv9cASYco3kC9q3D,47,...,computer blue,https://www.musixmatch.com/lyrics/Prince/Compu...,"Wendy?\nYes, Lisa?\nIs the water warm enough?\...",0,0,0.318073,0,380,en,48
2,GBASQ8200244,"['pop', 'power-pop', 'rock']",beyond belief,elvis costello & the attractions,Imperial Bedroom,1982,year,spotify:track:5qIWEtVAzylkR5zzMEk5uA,5qIWEtVAzylkR5zzMEk5uA,48,...,beyond belief,https://www.musixmatch.com/lyrics/Elvis-Costel...,History repeats the old conceits\nThe glib rep...,0,0,0.297604,0,1450,en,155
3,USMC17516873,['funk'],sweet thing,"rufus & chaka khan,chaka khan",The Very Best Of,1982-01-01,day,spotify:track:69eoS6bEWWfMac4TL36UiM,69eoS6bEWWfMac4TL36UiM,48,...,sweet thing,https://www.musixmatch.com/lyrics/RU%CC%88FU%C...,I will love you anyway\nEven if you cannot sta...,0,0,0.293638,0,1508,en,95
4,GBASQ8600339,"['pop', 'power-pop', 'rock']",i want you,elvis costello & the attractions,Blood And Chocolate,1986,year,spotify:track:1VN8ew9KECPqYS3sVbnZRr,1VN8ew9KECPqYS3sVbnZRr,46,...,i want you,https://www.musixmatch.com/lyrics/Elvis-Costel...,Oh my baby baby I love you more than I can tel...,0,0,0.287380,0,2029,en,180


In [9]:
# Filter out English lyrics
df = df[df['language'] != 'en']
print("Dataset after filtering non-English lyrics:")
df.head()


Dataset after filtering non-English lyrics:


,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,popularity,...,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank,lyrics_length,language,unique_words
6,DEA629260600,"['german', 'rock']",blaue augen,ideal,Ideal,1980,year,spotify:track:2pCPSNMAsw0R5FQkkvr6ry,2pCPSNMAsw0R5FQkkvr6ry,47,...,blaue augen,https://www.musixmatch.com/lyrics/Ideal/Blaue-...,Ideal und TV\nLässt mich völlig kalt\nUnd die ...,48,1,0.281598,53,1515,de,101
14,US78T0505165,['salsa'],penas,bonny cepeda,El Mandamás,1983-01-01,day,spotify:track:2D0bKs02ZVkDgmpvwR0CmT,2D0bKs02ZVkDgmpvwR0CmT,49,...,penas,https://www.musixmatch.com/lyrics/Bonny-Cepeda...,Pena\nEs lo que siento en mi alma\nPorque esa ...,0,0,0.263449,0,2154,es,184
19,USDBB0600177,['salsa'],detalles,orquesta inmensidad,La Salsa de Hoy,1984-01-01,day,spotify:track:2HdyWXCMDJw7TqAfR4A7Pt,2HdyWXCMDJw7TqAfR4A7Pt,48,...,detalles,https://www.musixmatch.com/lyrics/Orquesta-Inm...,"Na na na, na na\nUh uh\n\nYa no haces las cosi...",0,0,0.257062,0,1666,es,137
20,QMFMG1594099,['salsa'],poquita fé,roberto blades,Juntos Pa' Gozá,1988-09-03,day,spotify:track:5W5OkxGGVUNp5Sijs0fMG4,5W5OkxGGVUNp5Sijs0fMG4,48,...,poquita fé,https://www.musixmatch.com/lyrics/Roberto-Blad...,Yo sé que siempre dudas de mi amor\nY no te cu...,0,0,0.257015,0,1660,es,116
24,USAK60610039,['salsa'],señora ley,"conjunto clásico,tito nieves",Llego La Ley,1987-06-06,day,spotify:track:0yMyFYZ12CMqo59GG2bJzY,0yMyFYZ12CMqo59GG2bJzY,47,...,señora ley,https://www.musixmatch.com/lyrics/Conjunto-Cl%...,¿Ya no sé qué me pasa?\nSeñora Ley\n\nSe me ag...,0,0,0.252360,0,2582,es,162


In [10]:
# Define preprocessing functions


def preprocess_text(text):
    
    text = contractions.fix(text.lower())  
    text = re.sub(r'[^\w\s]', '', text)  
    tokens = word_tokenize(text)
    return ' '.join(tokens)





In [11]:
# Apply preprocessing functions
df['processed_lyrics'] = df['lyrics'].apply(preprocess_text)
print("Dataset with processed lyrics:")
df[['language', 'lyrics', 'processed_lyrics']].head()


Dataset with processed lyrics:


,language,lyrics,processed_lyrics
6,de,Ideal und TV\nLässt mich völlig kalt\nUnd die ...,ideal und tv lässt mich völlig kalt und die ga...
14,es,Pena\nEs lo que siento en mi alma\nPorque esa ...,pena es lo que siento en mi alma porque esa mu...
19,es,"Na na na, na na\nUh uh\n\nYa no haces las cosi...",na na na na na uh uh ya no haces las cositas q...
20,es,Yo sé que siempre dudas de mi amor\nY no te cu...,yo sé que siempre dudas de mi amor y no te cul...
24,es,¿Ya no sé qué me pasa?\nSeñora Ley\n\nSe me ag...,ya no sé qué me pasa señora ley se me agotan l...


In [12]:
# Translation function 
def translate_text_google(text, target_language='en'):
    api_key = ''  
    url = "https://translation.googleapis.com/language/translate/v2"
    params = {'q': text, 'target': target_language, 'key': api_key}
    response = requests.get(url, params=params)
    return response.json()['data']['translations'][0]['translatedText'] if response.status_code == 200 else text

df['translated_lyrics'] = df['lyrics'].apply(lambda x: translate_text_google(x, 'en'))
print("Dataset with translated lyrics:")
df[['language', 'lyrics', 'translated_lyrics']].head()


Dataset with translated lyrics:


,language,lyrics,translated_lyrics
6,de,Ideal und TV\nLässt mich völlig kalt\nUnd die ...,Ideal and TV Leaves me completely cold And the...
14,es,Pena\nEs lo que siento en mi alma\nPorque esa ...,Grief Is what I feel in my soul Because that w...
19,es,"Na na na, na na\nUh uh\n\nYa no haces las cosi...","Na na na, na na Uh uh You no longer do the lit..."
20,es,Yo sé que siempre dudas de mi amor\nY no te cu...,I know that you always doubt my love And I don...
24,es,¿Ya no sé qué me pasa?\nSeñora Ley\n\nSe me ag...,I don&#39;t know what&#39;s wrong with me anym...


In [14]:

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))



def summarize_text(text, max_sentences=5):
    sentences = sent_tokenize(text)
    return ' '.join(sentences[:max_sentences]) if len(sentences) > max_sentences else text

def handle_elongation_contraction(text):
    text = contractions.fix(text.lower())
    text = re.sub(r'(\w)\1+', r'\1\1', text)
    return text

def preprocess_translated_text(text):
    text = text.replace("&#39;", "'")
    text = handle_elongation_contraction(text)
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    # tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    text = ' '.join(tokens)
    return summarize_text(text)
    # # Convert HTML entities
    # text = html.unescape(text)

    # Handle elongations and contractions
    # text = handle_elongation_contraction(text)

    # # Tokenize and lower case
    # tokens = word_tokenize(text.lower())

    # # Handling repetition: Only keep words that are not excessively repeated
    # word_counts = Counter(tokens)
    # tokens = [word for word in tokens if word_counts[word] < 5]

    # # Remove stopwords and meaningless words
    # stop_words = set(stopwords.words('english'))
    # tokens = [word for word in tokens if word not in stop_words and len(word) > 1]

    # # Optionally, you can further clean the text here (e.g., remove non-alphanumeric characters)
    # text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [16]:

df['processed_translated_lyrics'] = df['translated_lyrics'].apply(preprocess_translated_text)
print("Dataset with processed translated lyrics:")
df[['language', 'translated_lyrics', 'processed_translated_lyrics']].head()

Dataset with processed translated lyrics:


,language,translated_lyrics,processed_translated_lyrics
6,de,Ideal and TV Leaves me completely cold And the...,ideal tv leaf completely cold whole scene hang...
14,es,Grief Is what I feel in my soul Because that w...,grief feel soul woman understand make lose coo...
19,es,"Na na na, na na Uh uh You no longer do the lit...",na na na na na uh uh longer little thing longe...
20,es,I know that you always doubt my love And I don...,know always doubt love blame know managed make...
24,es,I don&#39;t know what&#39;s wrong with me anym...,know wrong anymore mr law hope running punishm...


In [17]:
# Sentiment analysis
df['polarity'] = df['translated_lyrics'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['subjectivity'] = df['translated_lyrics'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
print("Dataset with sentiment analysis:")
df[['translated_lyrics', 'polarity', 'subjectivity']].head()


Dataset with sentiment analysis:


,translated_lyrics,polarity,subjectivity
6,Ideal and TV Leaves me completely cold And the...,0.054583,0.519583
14,Grief Is what I feel in my soul Because that w...,0.116839,0.571859
19,"Na na na, na na Uh uh You no longer do the lit...",0.050917,0.524000
20,I know that you always doubt my love And I don...,0.242527,0.588043
24,I don&#39;t know what&#39;s wrong with me anym...,0.277739,0.602963


In [18]:
# Emotion analysis
def analyze_emotion(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    doc = " ".join([token.lemma_ for token in doc])
    emotion_scores = NRCLex(doc).affect_frequencies
    return {f"{emotion}": score for emotion, score in emotion_scores.items()}

# Apply emotion analysis
df['emotions'] = df['processed_translated_lyrics'].apply(analyze_emotion)
print("Dataset with emotion analysis:")
df[['processed_translated_lyrics', 'emotions']].head()


Dataset with emotion analysis:


,processed_translated_lyrics,emotions
6,ideal tv leaf completely cold whole scene hang...,"{'fear': 0.07692307692307693, 'anger': 0.05494..."
14,grief feel soul woman understand make lose coo...,"{'fear': 0.07272727272727272, 'anger': 0.05454..."
19,na na na na na uh uh longer little thing longe...,"{'fear': 0.1259259259259259, 'anger': 0.022222..."
20,know always doubt love blame know managed make...,"{'fear': 0.06666666666666667, 'anger': 0.06666..."
24,know wrong anymore mr law hope running punishm...,"{'fear': 0.15384615384615385, 'anger': 0.07287..."


In [19]:
# Flatten emotions into separate columns
emotions_df = df['emotions'].apply(pd.Series)
emotions_df = emotions_df.loc[:, (emotions_df != 0).any(axis=0)]
df = pd.concat([df, emotions_df], axis=1).drop('emotions', axis=1)
print("Dataset with flattened emotions:")
df.head()


Dataset with flattened emotions:


,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,popularity,...,fear,anger,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
6,DEA629260600,"['german', 'rock']",blaue augen,ideal,Ideal,1980,year,spotify:track:2pCPSNMAsw0R5FQkkvr6ry,2pCPSNMAsw0R5FQkkvr6ry,47,...,0.076923,0.054945,0.054945,0.043956,0.142857,0.142857,0.208791,0.054945,0.065934,0.153846
14,US78T0505165,['salsa'],penas,bonny cepeda,El Mandamás,1983-01-01,day,spotify:track:2D0bKs02ZVkDgmpvwR0CmT,2D0bKs02ZVkDgmpvwR0CmT,49,...,0.072727,0.054545,0.090909,0.060606,0.193939,0.121212,0.084848,0.060606,0.151515,0.109091
19,USDBB0600177,['salsa'],detalles,orquesta inmensidad,La Salsa de Hoy,1984-01-01,day,spotify:track:2HdyWXCMDJw7TqAfR4A7Pt,2HdyWXCMDJw7TqAfR4A7Pt,48,...,0.125926,0.022222,0.051852,0.037037,0.192593,0.162963,0.148148,0.014815,0.177778,0.066667
20,QMFMG1594099,['salsa'],poquita fé,roberto blades,Juntos Pa' Gozá,1988-09-03,day,spotify:track:5W5OkxGGVUNp5Sijs0fMG4,5W5OkxGGVUNp5Sijs0fMG4,48,...,0.066667,0.066667,0.111111,0.066667,0.205556,0.127778,0.066667,0.055556,0.155556,0.077778
24,USAK60610039,['salsa'],señora ley,"conjunto clásico,tito nieves",Llego La Ley,1987-06-06,day,spotify:track:0yMyFYZ12CMqo59GG2bJzY,0yMyFYZ12CMqo59GG2bJzY,47,...,0.153846,0.072874,0.137652,0.016194,0.157895,0.194332,0.080972,0.036437,0.125506,0.024291


In [26]:
df['processed_translated_lyrics'] = df['processed_translated_lyrics'].fillna('').astype(str)
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(df['processed_translated_lyrics'])]

# Create and train Doc2Vec model
max_epochs = 10
vec_size = 168  
alpha = 0.025  

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                seed = 42,
                dm_mean = 1,
                min_count=1,
                dm=1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    
    model.alpha -= 0.0002
    
    model.min_alpha = model.alpha


model.save("d2v.model")
print("Model Saved")


def get_doc2vec_embeddings(text, model):
    return model.infer_vector(word_tokenize(text.lower()))

df['doc2vec_embeddings'] = df['processed_translated_lyrics'].apply(lambda x: get_doc2vec_embeddings(x, model))
print("Dataset with Doc2Vec embeddings:")
df[['processed_translated_lyrics', 'doc2vec_embeddings']].head()

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
Model Saved
Dataset with Doc2Vec embeddings:


,processed_translated_lyrics,doc2vec_embeddings
6,ideal tv leaf completely cold whole scene hang...,"[-0.37591228, 0.0092844805, -2.2214236, -0.032..."
14,grief feel soul woman understand make lose coo...,"[-0.10477852, 0.79797167, 0.5247491, 0.5611096..."
19,na na na na na uh uh longer little thing longe...,"[0.8478989, -0.40984017, -0.38339654, -0.78518..."
20,know always doubt love blame know managed make...,"[-0.40471104, 0.962982, -0.55453855, -0.460940..."
24,know wrong anymore mr law hope running punishm...,"[0.26561084, -0.7869881, -1.7856388, 0.6555000..."


In [27]:

embeddings_list = df['doc2vec_embeddings'].tolist()
embeddings_df = pd.DataFrame(embeddings_list)
embeddings_df.columns = ['doc2vec_embedding_' + str(col) for col in embeddings_df.columns]


df = pd.concat([df, embeddings_df], axis=1).drop('doc2vec_embeddings', axis=1)
print("Final dataset with Doc2Vec embeddings flattened:")
df.head()


Final dataset with Doc2Vec embeddings flattened:


,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,popularity,...,doc2vec_embedding_158,doc2vec_embedding_159,doc2vec_embedding_160,doc2vec_embedding_161,doc2vec_embedding_162,doc2vec_embedding_163,doc2vec_embedding_164,doc2vec_embedding_165,doc2vec_embedding_166,doc2vec_embedding_167
6,DEA629260600,"['german', 'rock']",blaue augen,ideal,Ideal,1980,year,spotify:track:2pCPSNMAsw0R5FQkkvr6ry,2pCPSNMAsw0R5FQkkvr6ry,47.0,...,0.558348,-0.510831,-1.136211,-1.329690,-0.343266,0.336933,1.279877,-0.369532,0.887167,0.631860
14,US78T0505165,['salsa'],penas,bonny cepeda,El Mandamás,1983-01-01,day,spotify:track:2D0bKs02ZVkDgmpvwR0CmT,2D0bKs02ZVkDgmpvwR0CmT,49.0,...,-0.165387,-0.670808,-0.771789,-0.309078,0.932054,0.901474,1.401674,1.499923,0.306798,0.641505
19,USDBB0600177,['salsa'],detalles,orquesta inmensidad,La Salsa de Hoy,1984-01-01,day,spotify:track:2HdyWXCMDJw7TqAfR4A7Pt,2HdyWXCMDJw7TqAfR4A7Pt,48.0,...,0.176868,0.145104,-0.464037,-1.011991,0.246744,0.725930,-0.221187,0.244322,0.332908,-0.743995
20,QMFMG1594099,['salsa'],poquita fé,roberto blades,Juntos Pa' Gozá,1988-09-03,day,spotify:track:5W5OkxGGVUNp5Sijs0fMG4,5W5OkxGGVUNp5Sijs0fMG4,48.0,...,-0.599124,-0.634924,-0.332354,0.003807,0.955686,0.978571,0.042352,-0.117470,0.741916,0.125501
24,USAK60610039,['salsa'],señora ley,"conjunto clásico,tito nieves",Llego La Ley,1987-06-06,day,spotify:track:0yMyFYZ12CMqo59GG2bJzY,0yMyFYZ12CMqo59GG2bJzY,47.0,...,-0.861992,-0.056941,-0.442984,-0.525238,0.544130,0.294607,-0.286385,-0.102570,0.080910,-0.112326


In [28]:


df.to_csv('processed_lyrics.csv', index=False)
print("Processed DataFrame saved to 'processed_lyrics.csv'.")


Processed DataFrame saved to 'processed_lyrics.csv'.
